# 05: Human-in-the-Loop

When the AI is uncertain, ask a human for help.

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from modules.m6_human_in_loop_v2 import (
    check_claim_with_human_review_v2,
    create_human_in_loop_graph_v2
)

# Visualize the graph
app = create_human_in_loop_graph_v2()
print("✅ Human-in-the-Loop BS Detector ready!")

✅ Human-in-the-Loop BS Detector ready!


## How It Works

1. **BS Detection** → Same as before
2. **Review Check** → If confidence < 60%, ask human
3. **Human Review** → (Simulated for demo)
4. **Final Output** → Human-validated result

In [2]:
# Test different confidence levels
test_claims = [
    "The Boeing 747 has four engines",  # High confidence
    "Scientists invented teleportation",  # Low confidence  
]

for claim in test_claims:
    print(f"\n📝 Claim: '{claim}'")
    result = check_claim_with_human_review_v2(claim)
    
    print(f"Verdict: {result['verdict']} ({result['confidence']}%)")
    print(f"Human Reviewed: {result.get('human_reviewed', False)}")


📝 Claim: 'The Boeing 747 has four engines'
Verdict: LEGITIMATE (100%)
Human Reviewed: False

📝 Claim: 'Scientists invented teleportation'
Verdict: BS (95%)
Human Reviewed: False


## Production Pattern

Replace the simulated review with real human input:

In [3]:
def production_human_review(state):
    """Real human review for production"""
    # Option 1: Queue for async review
    queue_id = queue_for_review(state)
    return {"status": "pending", "queue_id": queue_id}
    
    # Option 2: Direct user input
    # print(f"Review needed: {state['claim']}")
    # verdict = input("Verdict (BS/LEGITIMATE): ")
    # confidence = int(input("Confidence (0-100): "))
    # return {"verdict": verdict, "confidence": confidence}

# To use: replace simulate_human_review_node with production_human_review

## Key Benefits

1. **Simple Implementation** - Conditional routing, no interrupts
2. **Flexible Threshold** - Adjust when to ask for help
3. **Production Ready** - Easy to add real human review
4. **Improved Accuracy** - Humans handle uncertain cases

Next: Add memory to learn from human feedback!